In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
  #### imort libraries ####
import pandas as pd 
from pandas.core.frame import DataFrame 
import re #regular expretion    
import nltk  # natural language processing
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
  #sentement analysis 
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import string , spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS


from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
reader =pd.read_csv('/content/drive/MyDrive/nlp/Bitcoin_tweets.csv')
reader

,Unnamed: 0,date,Open_x,High_x,Low_x,Close_x,Volume_x,Unnamed: 0.1,Open_y,High_y,Low_y,Close_y,Volume_y,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,text,hashtags,source,is_retweet
0,87838,2021-02-05 10:53:00,37394.17,37401.05,37354.09,37389.76,41,87838.0,37394.17,37401.05,37354.09,37389.76,41.0,Nick Doevendans,"Edam-Volendam, Nederland","Amateur historicus m.n. WW2, schrijver, muziek...",2020-06-12 16:50:07,37.0,123.0,410.0,False,#reddcoin #rdd @reddcoin to the moon #altcoin ...,"['reddcoin', 'rdd', 'altcoin', 'turnreddcoinin...",Twitter for iPhone,False
1,87839,2021-02-05 10:54:00,37389.76,37416.07,37379.86,37380.00,28,87839.0,37389.76,37416.07,37379.86,37380.00,28.0,Nick Doevendans,"Edam-Volendam, Nederland","Amateur historicus m.n. WW2, schrijver, muziek...",2020-06-12 16:50:07,37.0,123.0,410.0,False,#reddcoin #rdd @reddcoin to the moon #altcoin ...,"['reddcoin', 'rdd', 'altcoin', 'turnreddcoinin...",Twitter for iPhone,False
2,87840,2021-02-05 10:55:00,37380.01,37436.53,37380.00,37425.34,21,87840.0,37380.01,37436.53,37380.00,37425.34,21.0,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,$PERL 0.06.\nI have insisted that since 0.02 i...,"['bitcoin', 'btc']",Twitter for Android,False
3,87841,2021-02-05 10:56:00,37425.35,37438.41,37395.91,37436.50,26,87841.0,37425.35,37438.41,37395.91,37436.50,26.0,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,$PERL 0.06.\nI have insisted that since 0.02 i...,"['bitcoin', 'btc']",Twitter for Android,False
4,87842,2021-02-05 10:57:00,37436.51,37450.00,37420.00,37436.26,29,87842.0,37436.51,37450.00,37420.00,37436.26,29.0,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,$PERL 0.06.\nI have insisted that since 0.02 i...,"['bitcoin', 'btc']",Twitter for Android,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436482,524320,2021-12-06 06:08:00,48734.90,48750.00,48721.57,48721.57,10,524320.0,48734.90,48750.00,48721.57,48721.57,10.0,Raj Patel,"Artesia, CA",Life is too short for bad vibes!,2012-01-28 23:06:05,1.0,6.0,9.0,False,Who thinks #Bitcoin investment is risky 🤔,['Bitcoin'],Twitter for Android,False
436483,524321,2021-12-06 06:09:00,48721.57,48753.55,48717.41,48738.11,14,524321.0,48721.57,48753.55,48717.41,48738.11,14.0,Raj Patel,"Artesia, CA",Life is too short for bad vibes!,2012-01-28 23:06:05,1.0,6.0,9.0,False,Who thinks #Bitcoin investment is risky 🤔,['Bitcoin'],Twitter for Android,False
436484,524322,2021-12-06 06:10:00,48738.12,48752.31,48700.01,48708.85,12,524322.0,48738.12,48752.31,48700.01,48708.85,12.0,Raj Patel,"Artesia, CA",Life is too short for bad vibes!,2012-01-28 23:06:05,1.0,6.0,9.0,False,Who thinks #Bitcoin investment is risky 🤔,['Bitcoin'],Twitter for Android,False
436485,524323,2021-12-06 06:11:00,48708.84,48712.20,48660.74,48696.41,11,524323.0,48708.84,48712.20,48660.74,48696.41,11.0,Raj Patel,"Artesia, CA",Life is too short for bad vibes!,2012-01-28 23:06:05,1.0,6.0,9.0,False,Who thinks #Bitcoin investment is risky 🤔,['Bitcoin'],Twitter for Android,False


In [4]:
btc=reader.loc[:,["date","text","user_location","user_description","user_followers","user_friends"]]


In [5]:
reader['Price Change']=reader['Close_y'].diff(periods=1)
reader.head()

,Unnamed: 0,date,Open_x,High_x,Low_x,Close_x,Volume_x,Unnamed: 0.1,Open_y,High_y,Low_y,Close_y,Volume_y,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,text,hashtags,source,is_retweet,Price Change
0,87838,2021-02-05 10:53:00,37394.17,37401.05,37354.09,37389.76,41,87838.0,37394.17,37401.05,37354.09,37389.76,41.0,Nick Doevendans,"Edam-Volendam, Nederland","Amateur historicus m.n. WW2, schrijver, muziek...",2020-06-12 16:50:07,37.0,123.0,410.0,False,#reddcoin #rdd @reddcoin to the moon #altcoin ...,"['reddcoin', 'rdd', 'altcoin', 'turnreddcoinin...",Twitter for iPhone,False,NaN
1,87839,2021-02-05 10:54:00,37389.76,37416.07,37379.86,37380.00,28,87839.0,37389.76,37416.07,37379.86,37380.00,28.0,Nick Doevendans,"Edam-Volendam, Nederland","Amateur historicus m.n. WW2, schrijver, muziek...",2020-06-12 16:50:07,37.0,123.0,410.0,False,#reddcoin #rdd @reddcoin to the moon #altcoin ...,"['reddcoin', 'rdd', 'altcoin', 'turnreddcoinin...",Twitter for iPhone,False,-9.76
2,87840,2021-02-05 10:55:00,37380.01,37436.53,37380.00,37425.34,21,87840.0,37380.01,37436.53,37380.00,37425.34,21.0,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,$PERL 0.06.\nI have insisted that since 0.02 i...,"['bitcoin', 'btc']",Twitter for Android,False,45.34
3,87841,2021-02-05 10:56:00,37425.35,37438.41,37395.91,37436.50,26,87841.0,37425.35,37438.41,37395.91,37436.50,26.0,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,$PERL 0.06.\nI have insisted that since 0.02 i...,"['bitcoin', 'btc']",Twitter for Android,False,11.16
4,87842,2021-02-05 10:57:00,37436.51,37450.00,37420.00,37436.26,29,87842.0,37436.51,37450.00,37420.00,37436.26,29.0,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,$PERL 0.06.\nI have insisted that since 0.02 i...,"['bitcoin', 'btc']",Twitter for Android,False,-0.24


In [6]:
reader = reader[reader['Price Change'].notna()]


In [7]:
def getAnalysis(score):
    if score < 0:
        return '0'
    
    else:
        return '1'
reader['Sentiment'] = reader['Price Change'].apply(getAnalysis )
reader.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Unnamed: 0,date,Open_x,High_x,Low_x,Close_x,Volume_x,Unnamed: 0.1,Open_y,High_y,Low_y,Close_y,Volume_y,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,text,hashtags,source,is_retweet,Price Change,Sentiment
1,87839,2021-02-05 10:54:00,37389.76,37416.07,37379.86,37380.00,28,87839.0,37389.76,37416.07,37379.86,37380.00,28.0,Nick Doevendans,"Edam-Volendam, Nederland","Amateur historicus m.n. WW2, schrijver, muziek...",2020-06-12 16:50:07,37.0,123.0,410.0,False,#reddcoin #rdd @reddcoin to the moon #altcoin ...,"['reddcoin', 'rdd', 'altcoin', 'turnreddcoinin...",Twitter for iPhone,False,-9.76,0
2,87840,2021-02-05 10:55:00,37380.01,37436.53,37380.00,37425.34,21,87840.0,37380.01,37436.53,37380.00,37425.34,21.0,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,$PERL 0.06.\nI have insisted that since 0.02 i...,"['bitcoin', 'btc']",Twitter for Android,False,45.34,1
3,87841,2021-02-05 10:56:00,37425.35,37438.41,37395.91,37436.50,26,87841.0,37425.35,37438.41,37395.91,37436.50,26.0,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,$PERL 0.06.\nI have insisted that since 0.02 i...,"['bitcoin', 'btc']",Twitter for Android,False,11.16,1
4,87842,2021-02-05 10:57:00,37436.51,37450.00,37420.00,37436.26,29,87842.0,37436.51,37450.00,37420.00,37436.26,29.0,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,$PERL 0.06.\nI have insisted that since 0.02 i...,"['bitcoin', 'btc']",Twitter for Android,False,-0.24,0
5,87843,2021-02-05 10:58:00,37436.26,37437.24,37406.21,37416.46,17,87843.0,37436.26,37437.24,37406.21,37416.46,17.0,🚀 AITeChain 🏹,Dubai - United Arab Emirates,loved Crypto since 2017\nفقط أضيف الحسابات الم...,2019-12-13 09:31:13,163.0,288.0,1908.0,False,#Amazing 😍\n#Monopoly #Crypto #cryptocurrency ...,"['Amazing', 'Monopoly', 'Crypto', 'cryptocurre...",Twitter for Android,False,-19.80,0


In [8]:
reader.Sentiment.value_counts()

0    219186
1    217300
Name: Sentiment, dtype: int64

In [9]:
import pandas as pd 
reader=pd.DataFrame(reader)
reader=reader.sample(n=100000)


In [10]:
reader.shape

(100000, 27)

In [11]:
############ \Drop Emoji from tweets ###########
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", flags = re.UNICODE)
    #print(regrex_pattern)
    return regrex_pattern.sub(r'', text)
    


reader['text'] = reader['text'].apply(deEmojify) 
reader

,Unnamed: 0,date,Open_x,High_x,Low_x,Close_x,Volume_x,Unnamed: 0.1,Open_y,High_y,Low_y,Close_y,Volume_y,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,text,hashtags,source,is_retweet,Price Change,Sentiment
198626,286464,2021-06-23 19:22:00,33390.38,33394.41,33357.15,33361.64,28,286464.0,33390.38,33394.41,33357.15,33361.64,28.0,🌋🌋 DeepBlueCrypto 🌋🌋,Satoshi Heights,Blockchain & Crypto enthusiast. Software. Lead...,2009-06-04 14:32:20,2316.0,351.0,29567.0,False,@mikemcglone11 $30k in2021 is the new $4k of 2...,['Bitcoin'],Twitter for iPhone,False,-33.40,0
116163,204001,2021-04-27 12:59:00,54812.38,54839.92,54809.90,54838.53,30,204001.0,54812.38,54839.92,54809.90,54838.53,30.0,📊EMILY CRYPTOCURRENCIES INVESTMENT📈📉,United Kingdom,Forex and binary trader📉📈/Account Manager 📊/ad...,2016-05-10 15:43:04,3584.0,121.0,7206.0,False,What we do here is #Bitcoin mining and binary ...,"['Bitcoin', 'btc']",Twitter for iPhone,False,26.15,1
49818,137656,2021-03-12 04:00:00,57107.14,57209.43,57070.42,57189.92,40,137656.0,57107.14,57209.43,57070.42,57189.92,40.0,Live Trading News,Off the 'chain,#Bitcoin #Doge #Ripple,2009-04-04 14:38:18,43357.0,40669.0,33689.0,False,"Europe is No Safe Haven, Buy Bitcoin, Dogecoin...","['Bitcoin', 'btc', 'Ethereum']",StockTwits Web,False,85.75,1
414108,501946,2021-11-20 17:14:00,57689.23,57709.92,57680.15,57709.87,15,501946.0,57689.23,57709.92,57680.15,57709.87,15.0,Raj Patel,"Artesia, CA",Life is too short for bad vibes!,2012-01-28 23:06:05,1.0,6.0,9.0,False,Who thinks #Bitcoin investment is risky,['Bitcoin'],Twitter for Android,False,20.65,1
99579,187417,2021-04-15 17:21:00,62437.51,62505.91,62413.95,62505.00,35,187417.0,62437.51,62505.91,62413.95,62505.00,35.0,Haylie lewis,"New York, USA",🌺🌺,2020-08-17 01:08:05,4.0,51.0,14.0,False,Make a change and an impart on yourself and ot...,"['bitcoin', 'cryptocurrency', 'crypto', 'block...",Twitter for iPhone,False,67.48,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121707,209545,2021-05-01 09:23:00,57411.70,57460.74,57411.70,57460.73,26,209545.0,57411.70,57460.74,57411.70,57460.73,26.0,📊EMILY CRYPTOCURRENCIES INVESTMENT📈📉,United Kingdom,Forex and binary trader📉📈/Account Manager 📊/ad...,2016-05-10 15:43:04,3584.0,121.0,7206.0,False,What we do here is #Bitcoin mining and binary ...,"['Bitcoin', 'btc']",Twitter for iPhone,False,49.02,1
127321,215159,2021-05-05 06:57:00,54318.02,54352.55,54318.01,54341.33,25,215159.0,54318.02,54352.55,54318.01,54341.33,25.0,📊EMILY CRYPTOCURRENCIES INVESTMENT📈📉,United Kingdom,Forex and binary trader📉📈/Account Manager 📊/ad...,2016-05-10 15:43:04,3584.0,121.0,7206.0,False,What we do here is #Bitcoin mining and binary ...,"['Bitcoin', 'btc']",Twitter for iPhone,False,23.31,1
51868,139706,2021-03-13 14:10:00,59716.96,59814.52,59716.96,59786.59,39,139706.0,59716.96,59814.52,59716.96,59786.59,39.0,CryptoInvest.Tech,Worldwide,https://t.co/gDTgo1OZVm provides you the Best ...,2019-04-13 19:28:05,407.0,8.0,21548.0,False,Altcoins sell-off after Bitcoin price rejects ...,['Bitcoin'],Twitter for Android,False,70.12,1
287886,375724,2021-08-24 23:32:00,48142.81,48142.81,48101.29,48112.28,14,375724.0,48142.81,48142.81,48101.29,48112.28,14.0,TechAdept (RDD),NYC,"Redd (RDD) Core Lead Dev, Project Redd Leader,...",2018-01-31 03:24:44,4903.0,322.0,5442.0,False,"As a constituent @RepAndyKimNJ, I urge you to ...","['cryptocurrency', 'DontKillCrypto', 'ReddCoin...",Twitter Web App,False,-29.04,0


In [12]:
features = reader.iloc[:, 21].values## text 
labels = reader.iloc[:, 26].values ## y : 1 

In [13]:
labels

array(['0', '1', '1', ..., '1', '0', '0'], dtype=object)

In [14]:
import nltk 
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
processed_features = []

for sentence in range(0, len(features)):
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))
    #remove rul: 
    processed_feature = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',processed_feature)
    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'#([^\s]+)', r'\1', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()
    # Lemmatization
    processed_feature = processed_feature.split()
    processed_feature = [ word for word in processed_feature if word not in stop_words and word not in punctuations ]
 
    processed_feature = [stemmer.lemmatize(word) for word in processed_feature]
    processed_feature = ' '.join(processed_feature)
    
    processed_features.append(processed_feature)

In [16]:
type(processed_feature)

str

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer (max_features=200, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
processed_features = vectorizer.fit_transform(processed_features).toarray()

In [18]:
processed_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [86]:
processed_features.shape

(100000, 200)

In [75]:
pd.DataFrame(processed_features)[0]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
99995    0.0
99996    0.0
99997    0.0
99998    0.0
99999    0.0
Name: 0, Length: 100000, dtype: float64

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)

In [89]:
''''
# SVR without kernel
from sklearn.svm import LinearSVR
lsvr = LinearSVR()
lsvr.fit(X_train, y_train)
print('Train score: {}'.format(lsvr.score(X_train, y_train)))
print('Test score: {}\n'.format(lsvr.score(X_test,y_test)))
'''


Train score: -0.7510059630433603
Test score: -0.782946815256865



In [20]:
from sklearn.linear_model import LogisticRegression
Classifier = LogisticRegression(random_state=0)
Classifier.fit(X_train, y_train)
print('Train score: {}'.format(Classifier.score(X_train, y_train)))
print('Test score: {}\n'.format(Classifier.score(X_test,y_test)))

### 51 % 
##  50 % 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Train score: 0.51155
Test score: 0.5029



In [21]:
from sklearn.naive_bayes import GaussianNB
Gauss = GaussianNB()
Gauss.fit(X_train, y_train)
#Y_pred = model.predict(X_test, y_test)
#model_score = model.score(X_test, y_test)
print('Train score: {}'.format(Gauss.score(X_train, y_train)))
print('Test score: {}\n'.format(Gauss.score(X_test,y_test)))

Train score: 0.50815
Test score: 0.50725



In [22]:
from sklearn.neighbors import KNeighborsClassifier
NN = KNeighborsClassifier()
NN.fit(X_train,y_train)

print('Train score: {}'.format(NN.score(X_train, y_train)))
print('Test score: {}\n'.format(NN.score(X_test,y_test)))

Train score: 0.5689375
Test score: 0.5



In [ ]:
from sklearn import svm
C = 1.0 
svc = svm.SVC(kernel ='linear', C = 1).fit(X_train, y_train)
print('Train score: {}'.format(svc.score(X_train, y_train)))
print('Test score: {}\n'.format(svc.score(X_test,y_test)))


### train : 51% 
##  test   : 50 %

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)
print('Train score: {}'.format(svc.score(X_train, y_train)))
print('Test score: {}\n'.format(svc.score(X_test,y_test)))

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf3 = RandomForestClassifier(max_depth=2, random_state=0)
clf3.fit(X_train,y_train )

print('Accuracy of Decision Tree classifier on training set: {:.2f}'
    .format(clf3.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
    .format(clf3.score(X_test, y_test)))

Accuracy of Decision Tree classifier on training set: 0.51
Accuracy of Decision Tree classifier on test set: 0.50


In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf3 = RandomForestClassifier(max_depth=4, random_state=0)
clf3.fit(X_train,y_train )

print('Accuracy of Decision Tree classifier on training set: {:.2f}'
    .format(clf3.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
    .format(clf3.score(X_test, y_test)))

Accuracy of Decision Tree classifier on training set: 0.52
Accuracy of Decision Tree classifier on test set: 0.50


#DROW